In [1]:
from os import listdir
import re
import chardet

In [43]:
def extract_details(file_text):
    re_patterns = [
        "Title: (.+)\n\nAuthor: ([^\n]+)\n",
        "  We need your donations.\n\n\n([^\n]+)\n\nby ([^\n]+)\n\n",
    ]

    title = None
    author = None

    for pattern in re_patterns:
        try:
            result = re.search(pattern, file_text, re.DOTALL)
            title = result.group(1)
            author = result.group(2)
            success += 1
            break
        except:
            continue
    if author == None or title == None:
        print(file_name + "******REGEX*************************************************")
        fail += 1
        regex_error += 1
    else:
        print(file_name)
        print("TITLE: " + title)
        print("AUTHOR: " + author + "\n")
    
    return title, author

In [59]:
txt_files = [f for f in listdir() if f.endswith(".txt")]

success = 0
fail = 0
encoding_error = 0
regex_error = 0

encoding_failed = []
regex_failed = []

for file_name in txt_files:
    
    with open(file_name, 'r') as file:
        #print(file_name)
        #print(file)
        #print(chardet.detect(file))
        try:
            file_text = file.read()
            
            re_patterns = [
                #"Title: (.+)\n\nAuthor: ([^\n]+)\n",
                "[ ]{0,4}Title: (.+)\n\n[ ]{0,4}Author: ([^\n]+)\n",
                "  We need your donations.\n\n\n([^\n]+)\n\nby ([^\n]+)\n\n",
                "\n\nTitle: (.+)\nAuthor: (.+)\nRelease Date: ",
                "\n\n\n\n\n\n[\d]{4}\n\n()\n\nby ()\n\n\nDramatis Personae",
            ]
            
            title = None
            author = None
            
            for pattern in re_patterns:
                try:
                    result = re.search(pattern, file_text, re.DOTALL)
                    title = result.group(1)
                    author = result.group(2)
                    success += 1
                    break
                except:
                    continue
            if author == None or title == None:
                print(file_name + "******REGEX*************************************************")
                fail += 1
                regex_error += 1
                regex_failed.append(file_name)
            else:
                print(file_name)
                print("TITLE: " + title)
                print("AUTHOR: " + author + "\n")
                
        except:
            print(file_name + "********ENCODING***********************************************")
            fail += 1
            encoding_error += 1
            encoding_failed.append(file_name)

print("Succes: " + str(success))
print("Fail: " + str(fail))
print("regex_errors: " + str(regex_error))
print("encoding_error: " + str(encoding_error))
print("Total: " + str(len(txt_files)))

20417.txt
TITLE: The Outline of Science, Vol. 1 (of 4)
       A Plain Story Simply Told
AUTHOR: J. Arthur Thomson

20770.txt********ENCODING***********************************************
21071.txt
TITLE: The Rover's Secret
       A Tale of the Pirate Cays and Lagoons of Cuba
AUTHOR: Harry Collingwood

21410.txt
TITLE: The Isle Of Pines (1668)
       and, An Essay in Bibliography by W. C. Ford
AUTHOR: Henry Neville

20198.txt
TITLE: Lavengro
       the Scholar - the Gypsy - the Priest

AUTHOR: George Borrow

20959.txt
TITLE: Mexico
       Its Ancient and Modern Civilisation, History, Political
       Conditions, Topography, Natural Resources, Industries and
       General Development
AUTHOR: Charles Reginald Enock

21430.txt
TITLE: Mexico and its Religion
       With Incidents of Travel in That Country During Parts of
       the Years 1851-52-53-54, and Historical Notices of Events
       Connected With Places Visited
AUTHOR: Robert A. Wilson

21426.txt
TITLE: The Tale of Daddy Longleg

20531.txt
TITLE: Up To Date Business
       Home Study Circle Library Series (Volume II.)
AUTHOR: Various

20318.txt
TITLE: Little Journeys to the Homes of the Great - Volume 14
       Little Journeys to the Homes of Great Musicians
AUTHOR: Elbert Hubbard

21320.txt
TITLE: Mass' George
       A Boy's Adventures in the Old Savannah
AUTHOR: George Manville Fenn

20487.txt
TITLE: Shakspere, Personal Recollections
AUTHOR: John A. Joyce

21329.txt
TITLE: The Nabob, Vol. 2 (of 2)
AUTHOR: Alphonse Daudet

21354.txt
TITLE: Menhardoc
AUTHOR: George Manville Fenn

21029.txt
TITLE: The New Morning
       Poems

AUTHOR: Alfred Noyes

20488.txt********ENCODING***********************************************
20485.txt
TITLE: The Lunatic at Large
AUTHOR: J. Storer Clouston

21052.txt
TITLE: The Pirate Shark
AUTHOR: Elliott Whitney

20201.txt
TITLE: Mary Gray
AUTHOR: Katharine Tynan

21465.txt
TITLE: Hurricane Hurry
AUTHOR: W.H.G. Kingston

21357.txt
TITLE: Nic Revel
       A White Slave's Adventures i

21227.txt
TITLE: Shenac's Work at Home
AUTHOR: Margaret Murray Robertson

21317.txt
TITLE: A Life's Eclipse
AUTHOR: George Manville Fenn

20800.txt
TITLE: London Lectures of 1907
AUTHOR: Annie  Besant

21403.txt
TITLE: The Pirate of the Mediterranean
       A Tale of the Sea
AUTHOR: W.H.G. Kingston

21358.txt
TITLE: The Ocean Cat's Paw
       The Story of a Strange Cruise
AUTHOR: George Manville Fenn

20967.txt
TITLE: The New Guide to Peterborough Cathedral
AUTHOR: George S. Phillips

21366.txt
TITLE: Sail Ho!
       A Boy at Sea
AUTHOR: George Manville Fenn

20172.txt
TITLE: The Flutter of the Goldleaf; and Other Plays
AUTHOR: Olive Tilford Dargan and Frederick Peterson

20146.txt
TITLE: Engineering Bulletin No 1: Boiler and Furnace Testing
AUTHOR: Rufus T. Strohm

20263.txt
TITLE: Boswell's Correspondence with the Honourable Andrew Erskine, and His Journal of a Tour to Corsica

AUTHOR: James Boswell

21062.txt
TITLE: The Cruise of the Nonsuch Buccaneer
AUTHOR: Harry Collingwood

2020

20737.txt
TITLE: Madge Morton's Secret

AUTHOR: Amy D. V. Chalmers

20608.txt
TITLE: How to Get on in the World
       A Ladder to Practical Success
AUTHOR: Major A.R. Calhoon

20429.txt
TITLE: The Seventh Noon
AUTHOR: Frederick Orin Bartlett

20527.txt
TITLE: The Sherwood Foresters in the Great War 1914 - 1919
       History of the 1/8th Battalion
AUTHOR: W.C.C. Weetman

21263.txt
TITLE: On the Old Road, Vol. 2 (of 2)
       A Collection of Miscellaneous Essays and Articles on Art and Literature
AUTHOR: John Ruskin

20158.txt********ENCODING***********************************************
21312.txt
TITLE: Glyn Severn's Schooldays
AUTHOR: George Manville Fenn

21270.txt
TITLE: Five Hundred Dollars
       or, Jacob Marlowe's Secret
AUTHOR: Horatio Alger

20247.txt
TITLE: Wayside Courtships
AUTHOR: Hamlin Garland

20755.txt
TITLE: Short Studies on Great Subjects
AUTHOR: James Anthony Froude

20400.txt
TITLE: On the Heels of De Wet

AUTHOR: The Intelligence Officer

20236.txt
TITLE: Fair t

20363.txt
TITLE: Continental Monthly , Vol. 5, No. 6, June, 1864
       Devoted to Literature and National Policy
AUTHOR: Various

20753.txt
TITLE: The Wonder Island Boys:  The Tribesmen
AUTHOR: Roger Finlay

21243.txt
TITLE: The Madigans
AUTHOR: Miriam Michelson

21395.txt
TITLE: The Last Look
       A Tale of the Spanish Inquisition
AUTHOR: W.H.G. Kingston

20896.txt
TITLE: Carry's Rose
       or, the Magic of Kindness. A Tale for the Young

AUTHOR: Mrs. George Cupples

21296.txt
TITLE: Mother Carey's Chicken
       Her Voyage to the Unknown Isle
AUTHOR: George Manville Fenn

20878.txt
TITLE: Critical Miscellanies, Vol. I
       Essay 2: Carlyle
AUTHOR: John Morley

20370.txt
TITLE: Rhymes of the East and Re-collected Verses
AUTHOR: John Kendall (AKA Dum-Dum)

21019.txt
TITLE: At the Deathbed of Darwinism
       A Series of Papers
AUTHOR: Eberhard Dennert

21383.txt
TITLE: Adventures in Australia
AUTHOR: W.H.G. Kingston

20655.txt
TITLE: The Emma Gees

AUTHOR: Herbert Wes McBride

20

21442.txt
TITLE: Your Plants
       Plain and Practical Directions for the Treatment of Tender
       and Hardy Plants in the House and in the Garden
AUTHOR: James Sheehan

20553.txt
TITLE: Out Around Rigel
AUTHOR: Robert H. Wilson

21418.txt
TITLE: The Eugenic Marriage, Volume IV. (of IV.)
       A Personal Guide to the New Science of Better Living and Better Babies
AUTHOR: Grant Hague

20801.txt
TITLE: Some Remains (hitherto unpublished) of Joseph Butler, LL.D.

AUTHOR: Joseph Butler

20474.txt
TITLE: Grace Harlowe's Fourth Year at Overton College
AUTHOR: Jessie Graham Flower

21235.txt
TITLE: The Maidens' Lodge
       None of Self and All of Thee, (In the Reign of Queen Anne)
AUTHOR: Emily Sarah Holt

20752.txt
TITLE: The Journal of Negro History, Volume 2, 1917
AUTHOR: Various

20929.txt
TITLE: Little Novels of Italy
       Madonna Of The Peach-Tree, Ippolita In The Hills, The
       Duchess Of Nona, Messer Cino And The Live Coal, The Judgment
       Of Borso
AUTHOR: Maurice Henry 

21455.txt
TITLE: Dick Cheveley
       His Adventures and Misadventures
AUTHOR: W. H. G. Kingston

21318.txt
TITLE: The Lost Middy
       Being the Secret of the Smugglers' Gap
AUTHOR: George Manville Fenn

21190.txt
TITLE: Expositions of Holy Scripture
       Second Corinthians, Galatians, and Philippians Chapters
       I to End. Colossians, Thessalonians, and First Timothy.
AUTHOR: Alexander Maclaren

21065.txt
TITLE: The Log of a Privateersman
AUTHOR: Harry Collingwood

20243.txt
TITLE: Dross

AUTHOR: Henry Seton Merriman

20217.txt
TITLE: The Stranger
       A Drama, in Five Acts

AUTHOR: August von Kotzebue

20203.txt
TITLE: Autobiography of Benjamin Franklin
AUTHOR: Benjamin Franklin

20362.txt
TITLE: Girls and Women
AUTHOR: Harriet E. Paine (AKA E. Chester}

20510.txt
TITLE: Ade's Fables
AUTHOR: George Ade

21368.txt
TITLE: The Silver Canyon
       A Tale of the Western Plains
AUTHOR: George Manville Fenn

20717.txt
TITLE: The Girl on the Boat
AUTHOR: Pelham Grenville Wodehouse


20444.txt
TITLE: Knickerbocker, or New-York Monthly Magazine, March 1844
       Volume 23, Number 3
AUTHOR: Various

20174.txt
TITLE: Freedom, Truth and Beauty
AUTHOR: Edward Doyle

20150.txt
TITLE: Notes on My Books
AUTHOR: Joseph Conrad

21223.txt
TITLE: The Carbonels
AUTHOR: Charlotte M. Yonge

20421.txt
TITLE: Proserpina, Volume 1
       Studies Of Wayside Flowers
AUTHOR: John  Ruskin

21114.txt
TITLE: Heroes of Modern Europe
AUTHOR: Alice Birkhead

20666.txt
TITLE: The Debs Decision
AUTHOR: Scott Nearing

21122.txt
TITLE: More about Pixie
AUTHOR: Mrs. George de Horne Vaizey

20493.txt
TITLE: Stories and Sketches
AUTHOR: Harriet S. Caswell

20660.txt
TITLE: The Faith of Islam
AUTHOR: Edward Sell

20357.txt
TITLE: Jerry
AUTHOR: Jean Webster

20430.txt
TITLE: The After-glow of a Great Reign
       Four Addresses Delivered in St. Paul's Cathedral

AUTHOR: A. F. Winnington Ingram

20916.txt
TITLE: The Arabian Nights
       Their Best-known Tales
AUTHOR: Unknown

20996.txt
TITLE: An Epo

21072.txt
TITLE: The Pirate Island
       A Story of the South Pacific
AUTHOR: Harry Collingwood

20282.txt
TITLE: The Working of Steel
       Annealing, Heat Treating and Hardening of Carbon and Alloy Steel
AUTHOR: Fred H. Colvin

21208.txt
TITLE: Among the Sioux
       A Story of the Twin Cities and the Two Dakotas
AUTHOR: R. J. Creswell

20652.txt
TITLE: Ring O' Roses
       A Nursery Rhyme Picture Book
AUTHOR: Anonymous

20312.txt
TITLE: Out of the Deep
       Words for the Sorrowful

AUTHOR: Charles Kingsley

20519.txt
TITLE: Highways in Hiding
AUTHOR: George Oliver Smith

20193.txt
TITLE: Mary's Rainbow

AUTHOR: Mary Edward Feehan

20538.txt
TITLE: Punch, or the London Charivari, Volume 104, February 4, 1893
AUTHOR: Various

21234.txt
TITLE: The Gold that Glitters
       The Mistakes of Jenny Lavender
AUTHOR: Emily Sarah Holt

20213.txt
TITLE: Peace on Earth, Good-will to Dogs
AUTHOR: Eleanor Hallowell Abbott

20202.txt
TITLE: Walnut Growing in Oregon
AUTHOR: Various

20758.txt
T

In [60]:
regex_failed

[]

In [51]:
ecoding_failed

['901.txt',
 '6080.txt',
 '5173.txt',
 '5243.txt',
 '4067.txt',
 '3960.txt',
 '5109.txt',
 '5709.txt',
 '4677.txt',
 '2250.txt',
 '666.txt',
 '2151.txt',
 '4770.txt',
 '660.txt',
 '664.txt',
 '3652.txt',
 '3073.txt',
 '141.txt',
 '5771.txt',
 '4277.txt',
 '735.txt',
 '2263.txt',
 '6597.txt',
 '5245.txt',
 '2253.txt',
 '2950.txt',
 '6577.txt',
 '1156.txt',
 '6579.txt',
 '2672.txt',
 '6460.txt',
 '2241.txt',
 '6685.txt',
 '4672.txt',
 '294.txt',
 '6735.txt',
 '7283.txt',
 '6334.txt',
 '580.txt',
 '4986.txt',
 '6473.txt',
 '4590.txt',
 '6493.txt',
 '1304.txt',
 '611.txt',
 '5605.txt',
 '5666.txt',
 '6671.txt',
 '6042.txt',
 '5351.txt',
 '4924.txt',
 '6568.txt',
 '4669.txt',
 '5620.txt',
 '3008.txt',
 '3611.txt',
 '3470.txt',
 '668.txt',
 '4997.txt',
 '6358.txt',
 '5636.txt',
 '3434.txt',
 '59.txt',
 '1589.txt',
 '5205.txt',
 '6868.txt',
 '3004.txt',
 '6896.txt',
 '3691.txt',
 '4761.txt',
 '6046.txt',
 '2448.txt',
 '6956.txt',
 '2578.txt',
 '3704.txt',
 '670.txt',
 '6923.txt',
 '2494.txt',